In [6]:
import sqlite3
import pandas as pd
pd.set_option('display.max_rows', None)

In [7]:
db = sqlite3.connect('Hop_Teaming_2018.sqlite')

In [9]:
with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    sqliteCursor = db.cursor()
    db.execute('CREATE INDEX IF NOT EXISTS Hop_Teaming_2018_from_npi ON Hop_Teaming_2018 (from_npi)')
    db.execute('CREATE INDEX IF NOT EXISTS Hop_Teaming_2018_to_npi ON Hop_Teaming_2018 (to_npi)')
    db.execute('CREATE UNIQUE INDEX IF NOT EXISTS nppes_npi ON nppes (npi)')
    db.execute('CREATE UNIQUE INDEX IF NOT EXISTS taxonomy_code ON taxonomy (code)')
    db.execute('CREATE UNIQUE INDEX IF NOT EXISTS zip_cbsa_unique ON zip_cbsa (zip,cbsa)')
    db.execute('CREATE INDEX IF NOT EXISTS zip_cbsa_zip ON zip_cbsa (zip)')
    db.execute('CREATE INDEX IF NOT EXISTS nppes_organization_name ON nppes (`provider_organization_name_(legal_business_name)`)')

## First, build a profile of providers referring patients to the major hospitals in Nashville. Are certain specialties more likely to refer to a particular hospital over the others?

To get major hospitals:  
1. join zip_cbsa table (cbsa 94180)
2. find the total number of referrals order by DESC
3. select top 10 / 15 entity with highest number of referrals

EXPLAIN QUERY PLAN - show estimated query run time 

In [28]:
#Rename old version of MONEY TABLE
#with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    #db.execute('ALTER TABLE hop_major_TN_hospitals_nores RENAME TO Hop_major_TN_hospitals_res_filter;')

In [26]:
# Create a new table that is a subset of Hop_Teaming_2018, our MONEY TABLE
query = '''
CREATE TABLE hop_major_TN_hospitals AS
SELECT hop.*
FROM Hop_Teaming_2018 AS hop
INNER JOIN nppes n1
ON hop.from_npi = n1.npi
INNER JOIN nppes n2
ON hop.to_npi = n2.npi 
INNER JOIN taxonomy as t
ON n1.primary_taxonomy = t.code
INNER JOIN taxonomy as t2
ON n2.primary_taxonomy = t2.code
INNER JOIN zip_cbsa as zip
ON n2.provider_business_practice_location_address_postal_code = zip.zip
WHERE n1.entity_type_code = 1
    AND n2.provider_business_practice_location_address_state_name = 'TN'
    AND n2.entity_type_code = 2
    AND t.specialization <> 'None'
    AND t2.classification LIKE '%Hospital'
    AND zip.cbsa = 34980;
'''

with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    db.execute(query)  
with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    db.execute('CREATE INDEX IF NOT EXISTS hop_major_TN_hospitals_from_npi ON hop_major_TN_hospitals (from_npi)')
    db.execute('CREATE INDEX IF NOT EXISTS hop_major_TN_hospitals_to_npi ON hop_major_TN_hospitals (to_npi)')

In [29]:
# View tables in database
con = sqlite3.connect('Hop_Teaming_2018.sqlite') 
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('nppes',), ('Hop_Teaming_2018',), ('taxonomy',), ('zip_cbsa',), ('Hop_major_TN_hospitals_res_filter',), ('hop_major_TN_hospitals',)]


In [17]:
# Find major hospitals
query = '''
SELECT t.specialization, n2.`provider_organization_name_(legal_business_name)`, 
       SUM(hop.transaction_count) AS total_transactions, SUM(hop.patient_count) AS total_patients
FROM Hop_Teaming_2018 AS hop
INNER JOIN nppes n1
ON hop.from_npi = n1.npi
INNER JOIN nppes n2
ON hop.to_npi = n2.npi 
INNER JOIN taxonomy as t
ON n1.primary_taxonomy = t.code
INNER JOIN taxonomy as t2
ON n2.primary_taxonomy = t2.code
INNER JOIN zip_cbsa as zip
ON n2.provider_business_practice_location_address_postal_code = zip.zip
WHERE n1.entity_type_code = 1
    AND n2.provider_business_practice_location_address_state_name = 'TN'
    AND n2.entity_type_code = 2
    AND t.specialization <> 'None'
    AND t2.classification LIKE '%Hospital'
    AND zip.cbsa = 34980
GROUP BY n2.`provider_organization_name_(legal_business_name)`
ORDER BY total_patients DESC;
'''

with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    Class_Hos_sqlite = pd.read_sql(query, db)
    
Class_Hos_sqlite

,specialization,provider_organization_name_(legal_business_name),total_transactions,total_patients
0,Family,VANDERBILT UNIVERSITY MEDICAL CENTER,485582,335156
1,Anatomic Pathology & Clinical Pathology,"HCA HEALTH SERVICES OF TENNESSEE, INC.",317471,219785
2,Diagnostic Radiology,SAINT THOMAS WEST HOSPITAL,231948,162537
3,Cardiovascular Disease,MAURY REGIONAL HOSPITAL,137679,92355
4,Cardiovascular Disease,HTI MEMORIAL HOSPITAL CORPORATION,73096,52344
5,Advanced Heart Failure and Transplant Cardiology,SAINT THOMAS RUTHERFORD HOSPITAL,68434,48344
6,Clinical Cardiac Electrophysiology,WILLIAMSON COUNTY HOSPITAL DISTRICT,66423,47434
7,Emergency Medical Services,SUMNER REGIONAL MEDICAL CENTER LLC,56606,37481
8,Family,HENDERSONVILLE HOSPITAL CORPORATION,46960,34818
9,Neurology,CENTRAL TENNESSEE HOSPITAL CORPORATION,29921,23127


In [33]:
# Find Major Hospitals. Same query as above but using the new MONEY TABLE to speed up run time. 
query = '''
SELECT n2.`provider_organization_name_(legal_business_name)`, 
       SUM(h.transaction_count) AS total_transactions, SUM(h.patient_count) AS total_patients
FROM hop_major_TN_hospitals AS h
INNER JOIN nppes n2
ON h.to_npi = n2.npi 
GROUP BY n2.`provider_organization_name_(legal_business_name)`
ORDER BY total_patients DESC;
'''

with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    Major_Hos_sqlite = pd.read_sql(query, db)
    
Major_Hos_sqlite

,provider_organization_name_(legal_business_name),total_transactions,total_patients
0,VANDERBILT UNIVERSITY MEDICAL CENTER,485582,335156
1,"HCA HEALTH SERVICES OF TENNESSEE, INC.",317471,219785
2,SAINT THOMAS WEST HOSPITAL,231948,162537
3,MAURY REGIONAL HOSPITAL,137679,92355
4,HTI MEMORIAL HOSPITAL CORPORATION,73096,52344
5,SAINT THOMAS RUTHERFORD HOSPITAL,68434,48344
6,WILLIAMSON COUNTY HOSPITAL DISTRICT,66423,47434
7,SUMNER REGIONAL MEDICAL CENTER LLC,56606,37481
8,HENDERSONVILLE HOSPITAL CORPORATION,46960,34818
9,CENTRAL TENNESSEE HOSPITAL CORPORATION,29921,23127


In [34]:
Major_Hos_sqlite.to_csv('Major_Hospitals.csv')

### First, build a profile of providers referring patients to the major hospitals in Nashville

In [36]:
# Specialization to Major Hospitals
query = '''
SELECT t.specialization, n2.`provider_organization_name_(legal_business_name)`, 
       SUM(h.transaction_count) AS total_transactions, SUM(h.patient_count) AS total_patients
FROM hop_major_TN_hospitals AS h
INNER JOIN nppes n1
ON h.from_npi = n1.npi
INNER JOIN nppes n2
ON h.to_npi = n2.npi 
INNER JOIN taxonomy as t
ON n1.primary_taxonomy = t.code
GROUP BY t.specialization, n2.`provider_organization_name_(legal_business_name)`
ORDER BY total_patients DESC;
'''

with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    Specialization_to_Hos_sqlite = pd.read_sql(query, db)
    
Specialization_to_Hos_sqlite

,specialization,provider_organization_name_(legal_business_name),total_transactions,total_patients
0,Diagnostic Radiology,"HCA HEALTH SERVICES OF TENNESSEE, INC.",101364,94621
1,Diagnostic Radiology,VANDERBILT UNIVERSITY MEDICAL CENTER,89513,81764
2,Diagnostic Radiology,SAINT THOMAS WEST HOSPITAL,74059,65059
3,Cardiovascular Disease,VANDERBILT UNIVERSITY MEDICAL CENTER,77918,58410
4,Diagnostic Radiology,MAURY REGIONAL HOSPITAL,44284,38907
5,Diagnostic Radiology,HTI MEMORIAL HOSPITAL CORPORATION,29490,27111
6,Anatomic Pathology & Clinical Pathology,"HCA HEALTH SERVICES OF TENNESSEE, INC.",35078,22779
7,Cardiovascular Disease,SAINT THOMAS WEST HOSPITAL,31117,22210
8,Diagnostic Radiology,SAINT THOMAS RUTHERFORD HOSPITAL,25063,21610
9,Diagnostic Radiology,WILLIAMSON COUNTY HOSPITAL DISTRICT,22734,20038


In [38]:
Specialization_to_Hos_sqlite.to_csv('Specialization_to_Major_Hospitals.csv')

### Are certain specialties more likely to refer to a particular hospital over the others?

In [44]:
# Get popularity ratio for each major hospital within a given specialization
query = '''
WITH specialization AS (
    SELECT t.specialization, SUM(h.transaction_count) AS total_transactions, SUM(h.patient_count) AS total_patients
    FROM hop_major_TN_hospitals AS h
    INNER JOIN nppes n1
    ON h.from_npi = n1.npi
    INNER JOIN taxonomy as t
    ON n1.primary_taxonomy = t.code
    GROUP BY t.specialization
)
SELECT t.specialization, n2.`provider_organization_name_(legal_business_name)`, 
       SUM(h.transaction_count) AS total_transactions, SUM(h.patient_count) AS total_patients,
       s.total_patients AS specialization_total_patients_ref,
       SUM(h.patient_count)*1.0/s.total_patients AS specialization_referral_ratio
FROM hop_major_TN_hospitals AS h
INNER JOIN nppes n1
ON h.from_npi = n1.npi
INNER JOIN nppes n2
ON h.to_npi = n2.npi 
INNER JOIN taxonomy as t
ON n1.primary_taxonomy = t.code
INNER JOIN specialization AS s
ON s.specialization = t.specialization 
GROUP BY t.specialization, n2.`provider_organization_name_(legal_business_name)`
ORDER BY t.specialization, specialization_referral_ratio DESC;
'''


with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    popularity_ratio_sqlite = pd.read_sql(query, db)
    
popularity_ratio_sqlite

,specialization,provider_organization_name_(legal_business_name),total_transactions,total_patients,specialization_total_patients_ref,specialization_referral_ratio
0,Acute Care,VANDERBILT UNIVERSITY MEDICAL CENTER,19828,13572,24337,0.557669
1,Acute Care,SAINT THOMAS WEST HOSPITAL,5989,4021,24337,0.165222
2,Acute Care,"HCA HEALTH SERVICES OF TENNESSEE, INC.",4525,3149,24337,0.129391
3,Acute Care,SAINT THOMAS RUTHERFORD HOSPITAL,1230,913,24337,0.037515
4,Acute Care,HTI MEMORIAL HOSPITAL CORPORATION,1192,784,24337,0.032214
5,Acute Care,MAURY REGIONAL HOSPITAL,1169,763,24337,0.031351
6,Acute Care,CENTRAL TENNESSEE HOSPITAL CORPORATION,501,415,24337,0.017052
7,Acute Care,HENDERSONVILLE HOSPITAL CORPORATION,262,218,24337,0.008958
8,Acute Care,SHELBYVILLE HOSPITAL COMPANY LLC,203,181,24337,0.007437
9,Acute Care,SUMNER REGIONAL MEDICAL CENTER LLC,196,168,24337,0.006903
